In [16]:
import pandas as pd
import pyodbc

In [17]:
df = pd.read_csv('for_isertion_df.csv')
df.head()

,Company,Job Title,Location,Job Type,Experience level,Requirment of the company,Facilities,AVG_Salary,Negotiation,asterisk,Country,job role
0,SGS,clinical data analyst,"['richardson', 'tx', 'united states']",Full Time,Entry-level,"['computer science', 'data quality', 'genetics...",No Facilities,48000.0,True,True,united states,data Analyste
1,Ocorian,aml/cft & data analyst,"['ebne', 'mauritius']",Full Time,Entry-level,"['agile', 'data management', 'finance', 'secur...",No Facilities,48000.0,True,True,mauritius,data Analyste
2,Cricut,machine learning engineer,"['south jordan', 'ut', 'united states']",Full Time,Not-Specified,"['agile', 'architecture', 'aws', 'computer sci...",['Career development'],90000.0,True,True,united states,IA
3,Bosch Group,application developer & data analyst,"['nonantola', 'italy']",Full Time,Entry-level,"['engineering', 'industrial', 'oracle', 'power...",No Facilities,48000.0,True,True,italy,data Analyste
4,Publicis Groupe,data engineer full time (public sector) usa,"['arlington', 'va', 'united states']",Full Time,Mid-level,"['aws', 'azure', 'computer science', 'consulti...","['Flex hours', 'Flex vacation', 'Parental leav...",108000.0,True,False,united states,data engineer


In [18]:
df['job role'].value_counts().index

Index(['data scientist', 'data engineer', 'data Analyste', 'IA', 'big data',
       'other'],
      dtype='object')

### Conection avec base de données :

In [19]:
# Créer un connexion dans le database :
driver = 'SQL Server'
server = 'LAPTOP-6D8J0VI4\SQLEXPRESS'
database = 'Offer_demploi_DB_1'

conn = pyodbc.connect(f'''
                        DRIVER={driver};
                        SERVER={server};
                        DATABASE={database};
                        Trusted_Connection=yes
                       ''')
cursor = conn.cursor()
conn

### La creation de tables :

In [20]:
cursor.execute('''
                    CREATE TABLE [job_opportunity] (
            [Id] INT NOT NULL,
            [Company_id] INT NOT NULL,
            [Location_id] INT NOT NULL,
            [Country_id] INT NOT NULL,
            [Job_Type] VARCHAR(50) NOT NULL CHECK ([Job_Type] IN ('Full Time', 'Part Time')),
            [Experience_level] VARCHAR(50) NOT NULL CHECK ([Experience_level] IN ('Senior-level','Mid-level','Entry-level','Not-Specified','Executive-level')),
            [AVG_Salary] FLOAT NOT NULL,
            [Negotiation] BIT NOT NULL,
            [custom_asterisk] BIT NOT NULL,
            [job_role] VARCHAR(50) NOT NULL CHECK ([job_role] IN ('data scientist','data engineer','data Analyste','IA','big data','other')),
            CONSTRAINT [PK_job_opportunity] PRIMARY KEY CLUSTERED ([Id] ASC)
        )

        CREATE TABLE [Company] (
            [Id] INT NOT NULL,
            [Name] VARCHAR(100) NOT NULL,
            CONSTRAINT [PK_Company] PRIMARY KEY CLUSTERED ([Id] ASC)
        )

        CREATE TABLE [Location] (
            [Id] INT NOT NULL,
            [Full_location] VARCHAR(100) NOT NULL,
            CONSTRAINT [PK_Location] PRIMARY KEY CLUSTERED ([Id] ASC)
        )

        CREATE TABLE [Country] (
            [id] INT NOT NULL,
            [Country_name] VARCHAR(100) NOT NULL,
            CONSTRAINT [PK_Country] PRIMARY KEY CLUSTERED ([id] ASC)
        )

        CREATE TABLE [Requirment_of_the_company] (
            [Id] INT NOT NULL,
            [Name] VARCHAR(1000) NOT NULL,
            CONSTRAINT [PK_Requirment_of_the_company] PRIMARY KEY CLUSTERED ([Id] ASC)
        )

        CREATE TABLE [Facilities] (
            [Id] INT NOT NULL,
            [Name] VARCHAR(1000) NOT NULL,
            CONSTRAINT [PK_Facilities] PRIMARY KEY CLUSTERED ([Id] ASC)
        )

        CREATE TABLE [Requirment_bridge] (
            [job_opportunity_id] INT NOT NULL,
            [Requirment_of_the_company_id] INT NOT NULL,
            CONSTRAINT [FK_Requirment_bridge_job_opportunity_id] FOREIGN KEY([job_opportunity_id])
            REFERENCES [job_opportunity] ([Id])
        )

        CREATE TABLE [Facilities_of_the_job] (
            [job_opportunity_id] INT NOT NULL,
            [Facility_id] INT NOT NULL,
            CONSTRAINT [FK_Facilities_of_the_job_job_opportunity_id] FOREIGN KEY([job_opportunity_id])
            REFERENCES [job_opportunity] ([Id])
        )

        ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Company_id] FOREIGN KEY([Company_id])
        REFERENCES [Company] ([Id])

        ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Company_id]

        ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Location_id] FOREIGN KEY([Location_id])
        REFERENCES [Location] ([Id])

        ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Location_id]

        ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Country_id] FOREIGN KEY([Country_id])
        REFERENCES [Country] ([id])

        ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Country_id]

        ALTER TABLE [Requirment_bridge] WITH CHECK ADD CONSTRAINT [FK_Requirment_bridge_Requirment_of_the_company_id] FOREIGN KEY([Requirment_of_the_company_id])
        REFERENCES [Requirment_of_the_company] ([Id])

        ALTER TABLE [Requirment_bridge] CHECK CONSTRAINT [FK_Requirment_bridge_Requirment_of_the_company_id]

        ALTER TABLE [Facilities_of_the_job] WITH CHECK ADD CONSTRAINT [FK_Facilities_of_the_job_Facility_id] FOREIGN KEY([Facility_id])
        REFERENCES [Facilities] ([Id])

        ALTER TABLE [Facilities_of_the_job] CHECK CONSTRAINT [FK_Facilities_of_the_job_Facility_id]
    ''')

cursor.commit()

### Afficher les Tables :

In [21]:
# show all table :
tables = cursor.tables(tableType='TABLE')
for table_info in tables:
    table_name = table_info.table_name
    print(table_name)
    print('#'*20)

Company
####################
Country
####################
Facilities
####################
Facilities_of_the_job
####################
job_opportunity
####################
Location
####################
Requirment_bridge
####################
Requirment_of_the_company
####################
trace_xe_action_map
####################
trace_xe_event_map
####################


## Insertions dans les tables :

#### Company :

In [22]:
# create unique company name :
company_unique = df['Company'].unique().tolist()

# insert data into table Company :
for i in range(len(company_unique)):
    print(i, company_unique[i])
    cursor.execute('INSERT INTO [Company] VALUES (?,?)',i,company_unique[i])

0 SGS
1 Ocorian
2 Cricut
3 Bosch Group
4 Publicis Groupe
5 ServiceNow
6 Natixis in Portugal
7 Palo Alto Networks
8 CERN
9 NielsenIQ
10 YouGov
11 PlayStation Global
12 Canva
13 Issuu
14 DeepL
15 Printify
16 Visa
17 Talan
18 EX2 Outcoding
19 CGG
20 RCA
21 Block
22 QuintoAndar
23 Neo Cybernetica
24 Western Digital
25 Podium
26 Netcentric
27 Freeform
28 Opera
29 Tiendanube
30 Ask Media Group
31 AVIV Group
32 Mutt Data
33 Global Atlantic Financial Group
34 Imagen Technologies
35 Sunnova Energy International
36 EUROPEAN DYNAMICS
37 Gemini
38 OpenSea
39 Weave HQ
40 Monzo
41 Scale AI
42 Channel Factory
43 Confused.com
44 Riverflex
45 Bionic Services Ltd
46 Nielsen
47 Etsy
48 PatientPoint
49 Fleetio
50 Databricks
51 Twitch
52 Roosh
53 Instacart
54 Snowflake Inc.
55 SSENSE
56 Amazon.com
57 NBCUniversal
58 Stripe
59 Kuda Technologies Ltd
60 KPMG Australia
61 DoubleVerify
62 Pure Integration
63 SAP Fioneer
64 Appen
65 Cyberlogic
66 Experian
67 Peloton
68 Foxintelligence
69 Sia Partners
70 GRAIL
71

In [23]:
cursor.commit()

### Location :

In [24]:
# create unique Location name :
Location_list = df['Location']
Location_list = Location_list.tolist()
# gat unique location name :
Location_list = list(set(Location_list))

# insert data into table Location table :
for i in range(len(Location_list)):
    cursor.execute('INSERT INTO [Location] VALUES (?,?)',i,Location_list[i])
cursor.commit()

### Country :

In [25]:
country_list = df['Country'].tolist()
# get unique country :
unique_country = list(set(country_list))
# insert data into table Country table :
for i in range(len(unique_country)):
    print(i, unique_country[i])
    cursor.execute('INSERT INTO [Country] VALUES (?,?)',i,unique_country[i])
cursor.commit()

0 nigeria
1 costa rica
2 armenia
3 uruguay
4 new zealand
5 denmark
6 latvia
7 romania
8 peru
9 austria
10 israel
11 bosnia and herzegovina
12 india
13 macao
14 germany
15 mauritius
16 egypt
17 sri lanka
18 norway
19 south africa
20 luxembourg
21 italy
22 jordan
23 malaysia
24 united states
25 malta
26 united kingdom
27 japan
28 moldova, republic of
29 greece
30 serbia
31 brazil
32 argentina
33 singapore
34 poland
35 switzerland
36 cyprus
37 philippines
38 lithuania
39 bangladesh
40 gabon
41 croatia
42 zimbabwe
43 indonesia
44 panama
45 ukraine
46 zambia
47 france
48 hungary
49 uganda
50 pakistan
51 kenya
52 united arab emirates
53 albania
54 new caledonia
55 north macedonia
56 hong kong
57 portugal
58 morocco
59 australia
60 mexico
61 montserrat
62 ireland
63 holy see (vatican city state)
64 belgium
65 puerto rico
66 oman
67 lebanon
68 czech republic
69 vietnam
70 remote
71 georgia
72 bulgaria
73 slovenia
74 gibraltar
75 chile
76 asia
77 spain
78 slovakia
79 thailand
80 azerbaijan
81 e

### job_opportunity :

In [26]:
df.head(1)

job_opportunity  = df[['Company','Location','Country','Job Type','Experience level','AVG_Salary','Negotiation','asterisk','job role']]
job_opportunity.head()

# change Company colomn into id :
def company_fk(c):
    # get index from value from c in company_unique :
    return company_unique.index(c)
job_opportunity['Company'] = job_opportunity['Company'].apply(company_fk)

# change Location column into id :
def location_fk(l):
    # get index from value from l in location_unique :
    return Location_list.index(l)
job_opportunity['Location'] = job_opportunity['Location'].apply(location_fk)

# change Country column into id :
def country_fk(c):
    # get index from value from c in country_unique :
    return unique_country.index(c)
job_opportunity['Country'] = job_opportunity['Country'].apply(country_fk)

# change Negotiation column frome true to 1 and from False to 0 :
job_opportunity['Negotiation'] = job_opportunity['Negotiation'].apply(lambda x: 1 if x == True else 0)

# change asterisk column frome true to 1 and from False to 0 :
job_opportunity['asterisk'] = job_opportunity['asterisk'].apply(lambda x: 1 if x == True else 0)

job_opportunity.head()

for index, row in job_opportunity.iterrows():
    Company = row['Company']
    Location = row['Location']
    Country = row['Country']
    Job_Type = row['Job Type']
    Experience_level= row['Experience level']
    AVG_Salary = row['AVG_Salary']
    Negotiation = row['Negotiation']
    asterisk = row['asterisk']
    job_role = row['job role']
    print(index,Company,Location,Country,Job_Type,Experience_level,AVG_Salary,Negotiation,asterisk,job_role)
    cursor.execute('INSERT INTO [job_opportunity] VALUES (?,?,?,?,?,?,?,?,?,?)',
                   index,
                   Company,
                   Location,
                   Country,
                   Job_Type,
                   Experience_level,
                   AVG_Salary,
                   Negotiation,
                   asterisk,
                   job_role)

C:\Users\Youcode\AppData\Local\Temp\ipykernel_17116\288106992.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_opportunity['Company'] = job_opportunity['Company'].apply(company_fk)
C:\Users\Youcode\AppData\Local\Temp\ipykernel_17116\288106992.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_opportunity['Location'] = job_opportunity['Location'].apply(location_fk)
C:\Users\Youcode\AppData\Local\Temp\ipykernel_17116\288106992.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy

0 0 237 24 Full Time Entry-level 48000.0 1 1 data Analyste
1 1 867 15 Full Time Entry-level 48000.0 1 1 data Analyste
2 2 380 24 Full Time Not-Specified 90000.0 1 1 IA
3 3 495 21 Full Time Entry-level 48000.0 1 1 data Analyste
4 4 152 24 Full Time Mid-level 108000.0 1 0 data engineer
5 5 701 24 Full Time Senior-level 184000.0 1 0 data scientist
6 6 840 57 Full Time Entry-level 39000.0 1 1 data scientist
7 7 847 24 Full Time Senior-level 140000.0 1 0 data engineer
8 8 704 35 Full Time Entry-level 40000.0 1 1 data scientist
9 9 916 93 Full Time Not-Specified 80000.0 1 1 data Analyste
10 10 272 12 Full Time Entry-level 35000.0 1 1 big data
11 3 459 57 Full Time Senior-level 62000.0 1 1 data scientist
12 3 759 14 Full Time Entry-level 39000.0 1 1 data Analyste
13 5 701 24 Full Time Senior-level 159000.0 1 0 data scientist
14 11 359 24 Full Time Senior-level 97000.0 1 0 IA
15 12 845 59 Full Time Senior-level 92000.0 1 1 data Analyste
16 13 137 57 Full Time Not-Specified 48000.0 1 1 big data

### Facilities :

In [28]:

def to_list(fac) :
    if fac == 'No Facilities' :
        return fac.split('§')
    else :
        return eval(fac)
    
facilities = df['Facilities'].apply(to_list)
facilities = facilities.tolist()

# change list in list to one list :
facilities = [item for sublist in facilities for item in sublist]
facilities = list(set(facilities))
# remove the empty values in list :
facilities.remove('')
for i in range(len(facilities)) :
    print(i, facilities[i])
    cursor.execute('INSERT INTO [Facilities] VALUES (?,?)',i,facilities[i])


0 Conferences
1 Paid sabbatical
2 Yoga
3 Parental leave
4 Wellness
5 Cell phone stipend
6 Health care
7 Medical leave
8 Equity
9 Relocation support
10 Transparency
11 Competitive pay
12 Startup environment
13 Fitness / gym
14 Career development
15 Team events
16 Freelance
17 Snacks / Drinks
18 Pet friendly
19 Gear
20 Signing bonus
21 Flex vacation
22 Home office stipend
23 Fertility benefits
24 Salary bonus
25 Flex hours
26 Contract
27 Flexible spending account
28 401(k) matching
29 Flat hierarchy
30 Lunch / meals
31 No Facilities
32 Travel
33 Insurance
34 Unlimited paid time off


### Facilities_of_the_job :

In [29]:
facilites_in_job_op = df['Facilities'].tolist()

# get index of facilites :
for i in range(len(facilites_in_job_op)) :
# facilites_in_job_op[i]
    for f in facilities :
        if f in facilites_in_job_op[i] :
            # index of f :
            print(i,'=>',facilities.index(f))
            cursor.execute('INSERT INTO [Facilities_of_the_job] VALUES (?,?)',i,facilities.index(f))


0 => 31
1 => 31
2 => 14
3 => 31
4 => 3
4 => 21
4 => 25
4 => 34
5 => 8
5 => 11
5 => 14
5 => 25
5 => 28
6 => 19
7 => 7
7 => 12
7 => 14
7 => 24
8 => 6
8 => 7
8 => 14
9 => 12
9 => 14
10 => 14
11 => 25
12 => 15
13 => 8
13 => 11
13 => 14
13 => 25
13 => 28
14 => 6
14 => 11
14 => 14
14 => 24
15 => 3
15 => 12
15 => 21
15 => 22
15 => 25
16 => 6
16 => 8
16 => 11
16 => 33
17 => 14
18 => 8
18 => 9
18 => 13
18 => 14
18 => 22
19 => 14
19 => 25
20 => 31
21 => 14
21 => 15
22 => 31
23 => 6
23 => 14
23 => 25
24 => 31
25 => 6
26 => 14
26 => 21
27 => 13
28 => 6
28 => 12
28 => 14
28 => 25
29 => 12
30 => 31
31 => 6
31 => 13
31 => 14
31 => 21
31 => 25
32 => 6
32 => 8
32 => 14
32 => 21
32 => 25
33 => 9
33 => 14
34 => 6
34 => 11
34 => 14
35 => 31
36 => 31
37 => 12
37 => 14
37 => 25
38 => 21
38 => 25
39 => 13
39 => 14
39 => 19
39 => 21
39 => 25
40 => 8
40 => 11
40 => 14
40 => 23
40 => 28
41 => 12
41 => 24
42 => 14
43 => 14
43 => 15
44 => 14
45 => 8
45 => 11
45 => 14
45 => 24
46 => 8
47 => 31
48 => 0
48 => 14
48 

### Requirment_of_the_company :

In [30]:
Requirment = df['Requirment of the company '].apply(eval)
Requirment = Requirment.tolist()

# change list in list to one list :
Requirment = [item for sublist in Requirment for item in sublist]
Requirment = list(set(Requirment))

# remove the empty values in list :
Requirment.remove('')

for i in range(len(Requirment)) :
    print(i, Requirment[i])
    # cursor.execute(f"INSERT INTO [Country] VALUES({int(i)},{str(Requirment[i])})")
    cursor.execute('INSERT INTO [Requirment_of_the_company] VALUES (?,?)',i,Requirment[i])


0 vr
1 knime
2 pandas
3 r
4 csv
5 power bi
6 r&d
7 keras
8 api development
9 dynamodb
10 consulting firm
11 php
12 data analytics
13 javascript
14 c++
15 unstructured data
16 crypto
17 recommender systems
18 genetics
19 industrial
20 kanban
21 machine learning
22 teradata
23 robotics
24 agile
25 diffusion models
26 coreml
27 autoregressive models
28 aws
29 metabase
30 pyspark
31 clojure
32 chatbots
33 matlab
34 physics
35 fraud risk
36 autonomous driving
37 ec2
38 caffe
39 datarobot
40 drones
41 ocr
42 vue
43 arrow
44 qlikview
45 model deployment
46 business analytics
47 biology
48 stem
49 flink
50 git
51 jax
52 pentaho
53 cad
54 ssis
55 github
56 clustering
57 okr
58 scala
59 computer science
60 scrum
61 perl
62 dataproc
63 cluster analysis
64 aistats
65 rdbms
66 rest api
67 nonprofit
68 mathematics
69 generative modeling
70 angular
71 data analysis
72 kpis
73 cudnn
74 causal inference
75 ai art
76 bigtable
77 redshift
78 google cloud
79 dicom
80 db2
81 qlik
82 dataflow
83 bigquery
84

### Requirment_bridge :

In [31]:
Requirment_bridge = df['Requirment of the company '].tolist()

# get index of facilites :
for i in range(len(Requirment_bridge)) :
# Requirment_bridge[i]
    for r in Requirment :
        if r in Requirment_bridge[i] :
            # index of f :
            print(i,'=>',Requirment.index(r))
            cursor.execute('INSERT INTO [Requirment_bridge] VALUES (?,?)',i,Requirment.index(r))

0 => 3
0 => 18
0 => 59
0 => 68
0 => 136
0 => 202
0 => 271
1 => 3
1 => 24
1 => 130
1 => 180
1 => 204
1 => 206
2 => 3
2 => 24
2 => 28
2 => 59
2 => 86
2 => 102
2 => 185
2 => 206
3 => 3
3 => 5
3 => 6
3 => 19
3 => 240
3 => 254
4 => 3
4 => 28
4 => 59
4 => 82
4 => 110
4 => 179
4 => 201
4 => 249
5 => 3
5 => 19
5 => 21
5 => 59
5 => 102
5 => 193
5 => 230
6 => 3
6 => 130
6 => 136
6 => 267
6 => 268
7 => 3
7 => 24
7 => 28
7 => 59
7 => 169
7 => 175
7 => 201
7 => 206
8 => 3
8 => 7
8 => 21
8 => 59
8 => 71
8 => 102
8 => 171
9 => 3
9 => 18
9 => 46
9 => 204
9 => 234
9 => 254
9 => 261
10 => 3
10 => 5
10 => 12
10 => 180
10 => 190
10 => 227
10 => 258
10 => 268
11 => 3
11 => 122
11 => 204
11 => 254
12 => 2
12 => 3
12 => 12
12 => 83
12 => 224
12 => 255
13 => 3
13 => 19
13 => 21
13 => 59
13 => 102
13 => 193
13 => 230
14 => 3
14 => 28
14 => 50
14 => 55
14 => 151
14 => 246
14 => 273
15 => 3
15 => 77
15 => 83
15 => 180
15 => 224
15 => 268
16 => 3
16 => 46
16 => 217
16 => 224
16 => 234
16 => 259
17 => 3
17 => 21
1

In [32]:
cursor.commit()
conn.close()